# GPT 

Мы немного разобрали и попробовали дообучать BERT для задачи классификации. В этой тетрадке посмотрим на другую популярную транформерную модель -  GPT (Generative pretrained transformer).

## Чем BERT и GPT похожи?
- Обе модели основаны на транформерных блоках с механизмом внимания и позиционным кодированием. Есть небольшие отличия в некоторых частях моделей (в gpt, например используется sparse attention), но они не принципиальные.

- В обоих моделях используется BPE токенизация. 
- Обе модели решают задачу языкового моделирования.
- В обеих моделях используется только self-attention. 

## Чем отличаются BERT и GPT?
- BERT - это энкодер модель. А GPT - это декодер модель.
- BERT обучается на задачах Мasked language modelling (MLM) и Next Sentence Prediction (NSP). Это искусственные self supervised задачи: в MLM модель должна заполнить пропуски в тексте (например, "Я [MASK] в [MASK] один." -> "Я пошел в кино один"), а NSP - определить являются ли два предложения соседними. А GPT - это авторегрессионная модель, которая учится продолжать данную ей последовательностью ("Я пошел.." -> "Я пошел в" -> "Я пошел в кино" -> "Я пошел в кино один")
- BERT смотрит на входной текст целиком, а в GPT attention для каждого отдельного токена расчитывается только по предудущим токенам.  
- В GPT (особенно в последней версии огромных размеров) целевой задачей является именно генерация текста и поэтому предобученную модель либо вообще никак не дообучают, либо дообучают на генерацию нужных текстов (а не конкретную задачу вроде классификации). BERT используется в основном только для эмбеддингов и дообучения на конкретные задачи (классификация), заполнение пропусков в практических задачах практически бесполезно. 

### Главная особенность GPT
Предобученную GPT можно использовать для решения целевой задачи (той же классификации) сразу из коробки, без дообучения, через генерацию текста. Нужно только сформулировать входную последовательность подходящим образом. Например, для определения класса текста можно подать в модель нужный текст и добавить в конце "Тема этого текста: ", на что модель выдаст продолжение "Новости" или "Спорт". Такой подход называется zero-shot learning. 
Еще есть few-shot learning, когда в модель подается что-то вроде "2+2=4, 1+3=4, 1+2= ", то есть модели показывается несколько примеров того, что от нее хотят и просят анологично продолжить последний пример. В этом случае также не происходит дообучения модели (веса не обновляются), поэтому на каждом запуске в модель нужно подавать нужный контекст. Такие контексты называются затравками (prompts). Подбирать их может быть достаточно сложно, даже есть такой термин "prompt engineering". 
Между fine-tuning'ом всей модели и zero/few-shot подходом есть еще целое семейство методов частичного дообучения, которые в целом называются адаптерами. Про них мы еще поговорим отдельно. (приминительно к промптам есть например метод который называется prompt tuning, Про него можно почитать вот тут - https://habr.com/ru/company/yandex/blog/588214/). 

Также важная характеристика GPT моделей - их огромный размер. Исследователи быстро заметили, что чем больше модель, тем выше качество. Причем качество прирастает не линейно, а экспоненциально и, кажется, есть даже минимальный размер, с которого начинают проявляться обобщающие свойства. Он находится где-то в районе 6-7 млрд параметров. Если посмотреть на размеры опенсорсных моделей (llama, stablelm, mistral, gemma), то обычно минимальный вариант - это как раз модель 6-7B. Есть даже такое направление исследований - scaling laws, где исследователи пытаются вывести формулы оптимального размера модели, оптимального количества данных и т.д. Но это все очень неточно и сложно, просто потому что обучать даже 1 модель очень долго и дорого, а для поиска оптимального размера нужно перебрать очень много вариантов (одна из самых известных статей про это - https://arxiv.org/abs/2203.15556, иногда можно увидеть выражения вроде chinchila optimal - это оттуда). 

Верхний предел для открытых моделей обычно находится в районе 60-70B, но это скорее hardware ограничение - дальше модели уже не помешаются на стандартные видеокарты даже с оптимизациями и требуется разворачивать целые кластеры. Про коммерческие модели мы знаем очень мало, но скорее всего их размер это сотни миллиардов параметров (про gpt-3 мы точно знаем, что это 175B, а про gpt-4 говорили, что ее размер - 1 триллион параметров, но это не точно). 

Есть варианты поменьше, в том числе и для русского (RuGPT-3 от Сбера). Разумеется их качество несравнимо c коммерческими моделями, но их можно попробовать в коллабе через huggingface, а принцип использования/обучения с размером не меняется, нужно просто больше вычеслительных ресурсов. 


**Важное уточнение:** мы будем пробовать базовые GPT модели, обученные только на генерации следующего токена. Про дообучение на инструкциях и human feedback поговорим на следующих семинарах.

In [1]:
# %pip install transformers

In [1]:
from transformers.utils import logging
# logging.set_verbosity(40)

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Загружаем модель ruGPT от сбера
model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

2024-03-07 09:49:16.336649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 09:49:17.130845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
# prompt engineering 
# text = "Вопрос: 'Сколько будет 2+2?' \n Ответ: " # работает
text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:" # не очень работает
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Input length of input_ids is 45, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.



 Вопрос: 'Сколько будет 3+3?' 
 Ответ: 6. 
 Вопрос: 'Сколько будет 1+9?' 
 Ответ: 10. 
 Вопрос: 'Сколько будет 4+2?' 
 Ответ: 12


## Методы генерации текста
В вводном семинаре про языковое моделирование мы уже пробовали генерировать текст и там мы сталкивались с вопросом - а как выбирать конкретный токен по вероятностям, которые выдает модель? Если просто использовать .argmax(), то генерация будет слишком детерменированной и будет много повторов. Обычно это не то, что нам нужно. GPT может применяться, например, для генерации ответов в чатботе и однобразность ответов - сильный минус.

На семинаре про базовое языковое моделирование мы решали это случайным семплированием из полученного распределения, а также пробовали писать свой beam search. Все это применимо и к gpt моделям. Стандартные методы уже реализованы в huggingface и остается лишь перебирать параметры.

In [9]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

### Argmax
Мы уже выше попробовали аргмакс, но давайте проверим его и на новом тексте, чтобы потом было с чем сравнивать

In [10]:
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




### Сэмплирование

Чтобы на каждом шаге выбирать случайное слово с учетом распределения, нужно просто поменять do_sample на True. Так результат будет меняться каждый раз

In [12]:
out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется спать, и спать<s>
Что делать, если невозможно войти в вк?
КАКСКОТ ГОВОРИТ- СИДИТЕ НА СЛУЖЕБНОМ ЗАВТ


Но случайный выбор может быть слишком случайным, так как иногда будут выбираться совсем маловероятные слова. Есть сразу несколько параметров, которые помогают контролировать случайность.

### top_k

Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов.
В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него. 


Чем больше top_k тем более случайный результат мы получим, но слишком низкий top_k может плохо сказаться на разнообразности.

In [13]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется плакать.

– Ты чего такая грустная, – спрашиваю я.

– Не знаю. – Я отворачиваюсь и смотрю в окно.

– Ты

### text with top_k -  10
За окном дождь. Холодный и противный. Хочется пить и есть. Я смотрю на часы и вижу, что у меня осталось еще несколько часов. Надо спешить и как можно быстрее добраться в аэропорт. Я иду на остановку. И вижу там,

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется пить. Я хватаюсь за стакан, но он не помогает.

В дверь постучали. В коридоре я замечаю девочку лет двенадцати.

— Простите, что отвлекаю

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется что-нибудь сделать, чтобы хоть как-то отогнать от себя эту промозглость. Наверное, я никогда не найду его. Пока не доберусь до цели.
Уже кот

### Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).

![](https://miro.medium.com/v2/resize:fit:1400/1*yJUw-zkpTfqdTVeOO7OKnQ.png)

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [15]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется закрыть окно и лечь спать. Но я не могу. Я не могу лечь спать. Я не могу лечь спать. Я не могу лечь спать. Я не могу лечь спать. Я не могу лечь

### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется закрыть глаза и закрыть уши.

— А у нас в квартире газ, — говорит мама.

— А у нас в квартире газ, — говорит папа.


### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется закрыть окно, но не могу. Не могу.

Я сижу на полу, прижавшись спиной к стене. Я слышу, как падает на пол капля, как шуршит шелковая нить. Я

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется завыть.

- Да, - говорю, - жизнь - она такая штука...

- Тю, - говорит, - чего ты!

- Не веришь? - говорю,

### text with temp -  1.0
За окном д

Видно, что чем больше температура, тем случайнее текст


Эти параметры можно совмещать, чтобы точнее найти границу между разнообразием и рандомом

In [16]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=0.9,
                     top_k=10,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется закрыть глаза и спрятаться под одеяло.

И все-таки я не хочу засыпать. Мне кажется, это неправильно. Неправильно, что я буду сидеть на кровати и тупо глядеть в


Температура большая, но текст все равно получается более менее тематически связанный

### Beam Search

У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). Напомню, что в нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации. 

In [18]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется закрыть глаза и ни о чем не думать. Но я не могу. Не могу. Не могу.

Я не могу. Не могу. Не могу. Не могу. Не могу. Не могу. Не могу. Не могу. Не могу


Но это тоже не очень сильно помогает

# Дообучение GPT

Чтобы генерировать тексты в нужном стиле/домене, можно дообучить GPT на своем корпусе. Давайте попробуем это сделать

In [19]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)


Для наглядности результата возьмем начало Евгения Онегина

In [20]:
text = """
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня.

Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.

Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.

Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант.
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.

Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale,
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.

Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.

Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.


Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!

Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!


Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


"""

В huggingface уже реализованы все инструменты для дообучения. Даже если вы не знакомы с pytorch разобраться будет не сложно

In [21]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/home/ubuntu/.pyenv/versions/3.10.4/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Обучение
Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [23]:
trainer.train()

wandb: Currently logged in as: manefedov26. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.11561222076416015, metrics={'train_runtime': 13.6393, 'train_samples_per_second': 139.303, 'train_steps_per_second': 7.332, 'total_flos': 62056857600000.0, 'train_loss': 0.11561222076416015, 'epoch': 100.0})

In [24]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
С севера на юг, 
Степь бурь и града 
За ним.

Как рано мог он хмуриться, 
Когда грозный рев 
Умом и страстью побеждал, 
И мог веселым быть, 
Когда сердце томило, 
И день и ночь ждал.

Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить,
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал.
Как рано мог он грустить, 
Когда сердце томило, 
И день и ночь ждал


Попробуем beam search

In [25]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
И по парку гуляет,
И в Летний сад идет,
И в Летний сад идет.

Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!
Как рано мог он так рано!


Модель скатывается в повторы и это плохо. Есть еще несколько параметров, которые помогают бороться с повторами.

### repetition_penalty
Этот параметр просто штрафует повторы. Чем он выше, тем сильнее штраф.

In [26]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
И по парку гуляет,
Когда еще не был.

Как рано мог он грустить!
Как рано мог он отчаиваться!
Как рано мог он терять надежду!
Как рано мог он отчаяваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаиваться!
Как рано мог он отчаи

### no_repeat_ngram_size
А этот параметр напрямую говорит, что нграммы такого размера не должны повторяться совсем

In [27]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
И стучится в окна. —
Где ты, мой ненаглядный?»
Он отвечал: «Я здесь».

Как рано мог он тревожить
Сердца любящих сердец!
Но вот настало утро,
Когда он явил наконец
Все свои надежды и мечты.
С ним был граф Бенкендорф;
Был Гейне, Ватопедский нищий…
Да здравствует гениальный Миклухо-Маклай![10 - М. В. Ломоносов (1769–1831) — русский ученый, один из основоположников естествознания.]
Высокой страсти не знал он
Ни к отечеству, ни к ученым спорам. Женился он на Софье Петровне Бестужевой-Рюминой, урожденной княжне Варшавской, дочери польского короля Сигизмунда I, а затем вдовы поэта Евгения Евграфовича Тургенева. Свадьба была пышная, но недолгая. Софья вышла замуж за отставного поручика лейб-гвардии Семеновского полка Ивана Яковлевича Шереметева, с которым познакомилась еще в Петербурге. У них родилось семеро детей: Борис, Дмитрий, Павел, Евгений, Михаил, Алексей, Федор, Иван, князь Болконский, прозванный Безбородко, Василий Долгорукий, Александр Невзоров, Константин Победоносец, В

Результат не очень хороший, но это самая маленькая модель.